In [1]:
import matplotlib.pyplot as plt
import random, shutil, pickle, sys
import numpy as np
from collections import deque
from keras import Sequential
from keras.optimizers import RMSprop
from IPython.display import display
from PIL import Image
import pandas as pd
from keras import Model
import keras, os, glob
import tensorflow as tf
from keras.layers import Layer, Dense, Conv2D, Flatten, RepeatVector,Dropout, LayerNormalization, MultiHeadAttention, GlobalAveragePooling1D
from keras.layers import Activation, LSTM, Bidirectional , Dropout
from keras import layers
# import cv2
# from sklearn.preprocessing import StandardScaler
import tensorflow as tf
# from sklearn.preprocessing import MinMaxScaler
import codecs
import csv
import secrets
import sqlite3
from IPython.display import display
from IPython.display import clear_output
import time
from tabulate import tabulate
import json
import datetime
from tensorflow.keras.optimizers import Adam
import math, datetime

Chart Decorator

In [2]:
class ChartDecorator:
    def __init__(self):
        self.balance_limit = 300
        self.standard_balance = 1000
        self.prev_trade = None
        
    def add_top_bottom_bar(self, img, draw_context):
        # Define the coor dinates of the black bar
        x1, y1 = 0, 204
        x2, y2 = 200, 224
        # Draw the black bar bottom bar
        draw_context.rectangle([x1, y1, x2, y2], fill=(0, 0, 0))
        # Draw the black bar top bar
        x1, y1 = 0, 0
        x2, y2 = 224, 14
        draw_context.rectangle([x1, y1, x2, y2], fill=(0, 0, 0))


    def draw_account_balance(self, img, draw_context, account_balance = 1000):
        balance = self.balance_limit + (account_balance - self.standard_balance)
        if balance == self.balance_limit:
            x1, y1 = 3, 204
            x2, y2 = 75 , 224
            # Draw the green bar for account
            draw_context.rectangle([x1, y1, x2, y2], fill=(110, 235, 131))

        elif balance < self.balance_limit:
            health_bar = int((balance * 75) / self.balance_limit )
            lost_bar = 75 - health_bar
            #issue 1
            # print(f"issue 1 balance {balance}")
            # print(f"health bar {health_bar}")
            
            # print(f"lost bar {lost_bar}")
            #current_balnace
            x1, y1 = 3, 204
            x2, y2 = health_bar, 224
            draw_context.rectangle([x1, y1, x2, y2], fill=(110, 235, 131))
            #lost bar
            x1, y1 = health_bar, 204
            x2, y2 = 75 , 224
            draw_context.rectangle([x1, y1, x2, y2], fill=(255, 0, 0))
        elif balance > self.balance_limit:
            x1, y1 = 3, 204
            x2, y2 = 75 , 224
            # Draw the green bar for account
            draw_context.rectangle([x1, y1, x2, y2], fill=(110, 235, 131)) 
            #Draw the profit section on account_bar
            profit = int(((balance - self.balance_limit) * 75) / self.balance_limit)

            x1, y1 = 80, 204
            x2, y2 = 80 + profit , 224
            # Draw the green bar for account
            draw_context.rectangle([x1, y1, x2, y2], fill=(0,0,255))# fill=(71, 44, 27)) 

        #draw start separator
        #draw Account Separator and end separator
        x1, y1 = 0, 204
        x2, y2 = 3 , 224
        # Draw the green bar for account
        draw_context.rectangle([x1, y1, x2, y2], fill=(255, 87, 20))     

        #draw Account Separator and end separator
        x1, y1 = 76, 204
        x2, y2 = 80 , 224
        # Draw the green bar for account
        draw_context.rectangle([x1, y1, x2, y2], fill=(255, 87, 20)) 

        #draw Account Separator and end separator
        x1, y1 = 155, 204
        x2, y2 = 160 , 224
        # Draw the green bar for account
        draw_context.rectangle([x1, y1, x2, y2], fill=(255, 87, 20)) 

    def draw_current_trade(self,img, draw_context, position=None):
        if position == "buy":
            x1, y1 = 160, 204
            x2, y2 = 175, 224
            draw_context.rectangle([x1, y1, x2, y2], fill=(255, 255, 0)) 
        elif position == "sell":
            x1, y1 = 180, 204
            x2, y2 = 195, 224
            draw_context.rectangle([x1, y1, x2, y2], fill=(0, 255, 0))  
        else:
            x1, y1 = 200, 204
            x2, y2 = 215, 224
            draw_context.rectangle([x1, y1, x2, y2], fill=(255,0,255)) 

        #Draw position separator icons
        x1, y1 = 175, 204
        x2, y2 = 180, 224
        draw_context.rectangle([x1, y1, x2, y2], fill=(255, 255, 255)) 
        x1, y1 = 195, 204
        x2, y2 = 200, 224
        draw_context.rectangle([x1, y1, x2, y2], fill=(255, 255, 255)) 
        x1, y1 = 215, 204
        x2, y2 = 220, 224
        draw_context.rectangle([x1, y1, x2, y2], fill=(255, 255, 255)) 

    def draw_profit_bar(self, img, draw_context, profit = 0, account_balance = 1000, position = None):
        pnl_parts = int(abs(profit) / 10)
        for segment in range(0, 200, 10):
            f = segment + 10
            x1, y1 = segment+10, 0
            x2, y2 = f + 1, 9
            draw_context.rectangle([x1, y1, x2, y2], fill=(255, 255, 255)) 
            #display profit on the chart
            if profit > 0 and pnl_parts > 0 and segment >= 100:
                if segment < pnl_parts * 10 + 100:
                    x1, y1 = segment, 0
                    x2, y2 = segment + 10, 9
                    draw_context.rectangle([x1, y1, x2, y2], fill=(27, 152, 224)) 
            elif profit < 0 and pnl_parts > 0 and segment <= 90 and segment >= 100 - abs(profit):           
                x1, y1 = segment, 0
                x2, y2 = segment + 10, 9
                draw_context.rectangle([x1, y1, x2, y2], fill=(255, 0, 0))  
            #draw progressive account balance below profit bar
            account_balance_bar = (200*account_balance)/2000
            x1, y1 = 0, 10
            x2, y2 = account_balance_bar, 14
            draw_context.rectangle([x1, y1, x2, y2], fill=(255, 119, 0)) 
            #Add a closing indication at the top of the 
            if position == "close" and profit > 0 and self.prev_trade[2] == 1:
                x1, y1 = 200, 0
                x2, y2 = 224, 14
                draw_context.rectangle([x1, y1, x2, y2], fill=(255, 255, 0))
            elif position == "close" and profit > 0 and self.prev_trade == 2:
                x1, y1 = 200, 0
                x2, y2 = 224, 14
                draw_context.rectangle([x1, y1, x2, y2], fill=(0, 255, 0))
            elif position == "close" and profit < 0 and self.prev_trade in [2,1]:
                x1, y1 = 200, 0
                x2, y2 = 224, 14
                draw_context.rectangle([x1, y1, x2, y2], fill=(255,0, 0))
       
    def draw_buy_bar(self, img2, start_lines, end_lines, position = None, profit = 0, prev_trade=None, account_balance = 1100 ):
        self.prev_trade = prev_trade
        
        from PIL import Image, ImageDraw
        d = ImageDraw.Draw(img2)
        
        self.add_top_bottom_bar(img2, d)
        self.draw_account_balance(img2, d, account_balance)

        if position == "sell" and 2 in [self.prev_trade[3]]:
            #draw icon for trade setup
            self.draw_current_trade(img2, d, "sell")
            fixed_ratio = False
            #issue fix bug 
            #a situation where there is profit but since the trade was entered there were pertuabations of zooming and changing of window prices indicated on the subwindow
            #fix this by giving a ratio use profit to check if we are profitable, then check if end lines are lower than start lines , use a ratio of 1px for 3 points
            #when scaling the bar check the distance remaining behind you at the top of the window. start from index 14 your calculations
            if profit >= 0 and start_lines["ask_line"] > end_lines["ask_line"] :
                # print(f"\nRatio bug found {end_lines}\n")
                ratio = 15 #int((start_lines["ask_line"] - end_lines["ask_line"] ) / 3)
                if end_lines["ask_line"] - ratio < 14:
                    start_lines["ask_line"] = 14
                    start_lines["bid_line"] = 18
                else:
                    start_lines["ask_line"] = end_lines["ask_line"] - ratio
                    start_lines["bid_line"] = end_lines["bid_line"] - ratio
                fixed_ratio = True

            # Draw indicator of trade start
            
            line_color = (200,200,200)
            top = (200, end_lines["ask_line"])
            bottom = (200, end_lines["bid_line"])
            d.line([top, bottom], fill=line_color, width=20)

            #Draw start marker for the trade
            line_color = (0, 255, 0)
            top = (200, start_lines["ask_line"])
            bottom = (200, start_lines["bid_line"])
            d.line([top, bottom], fill=line_color, width=30) 
            
            #Draw start line for trade entry
            line_color = (0, 255, 0)#(252, 81, 48)
            top = (50, start_lines["ask_line"])
            bottom = (50, start_lines["bid_line"])
            d.line([top, bottom], fill=line_color, width=250) 

            #Draw profit indicator of sell
            if profit < 0:
                #Draw the main bar of the trade position
                if fixed_ratio:
                    line_color = (255, int(ratio/2+10), 0)
                    top = (200, start_lines["ask_line"])
                    bottom = (200,end_lines["bid_line"])
                    d.line([top, bottom], fill=line_color, width=int(ration/5)+10)
                    fixed_ratio = False
                else:
                    line_color = (255, 0, 0)
                    top = (200, start_lines["ask_line"])
                    bottom = (200,end_lines["bid_line"])
                    d.line([top, bottom], fill=line_color, width=10)
                
            else:
                line_color = (0, 255, 0)
                top = (200, start_lines["ask_line"])
                bottom = (200,end_lines["bid_line"])
                d.line([top, bottom], fill=line_color, width=10)
            #Draw loss indicator for sell
            self.draw_profit_bar(img2, d, profit)
        elif position == "buy" and 1 in [self.prev_trade[3]]:
            #issue fix bug 2
            if profit >= 0 and end_lines["ask_line"] > start_lines["ask_line"]:
                ratio = 15
                if end_lines["bid_line"] + ratio > 204:
                    start_lines["ask_line"] = 200
                    start_lines["bid_line"] = 204
                else:
                    start_lines["ask_line"] = end_lines["ask_line"] + ratio
                    start_lines["bid_line"] = end_lines["bid_line"] + ratio         
            
            #draw bottom right trade status sell , buy , close
            self.draw_current_trade(img2, d, "buy") # 
            #Draw indicator of trade start
            #Draw end marker
            line_color = (65, 64, 102)
            top = (200, end_lines["ask_line"])
            bottom = (200, end_lines["bid_line"])
            d.line([top, bottom], fill=line_color, width=20) 

            #Draw start marker for buy
            line_color = (255, 255, 0)
            top = (200, start_lines["ask_line"])
            bottom = (200, start_lines["bid_line"])
            d.line([top, bottom], fill=line_color, width=30) 

            #Draw start buy line across the chart
            line_color = (255, 255, 0)#(255,0,200)
            top = (50, start_lines["ask_line"])
            bottom = (50, start_lines["bid_line"])
            d.line([top, bottom], fill=line_color, width=250)         


            if profit < 0:
                line_color = (255, 0, 0)
                top = (200, start_lines["ask_line"])
                bottom = (200, end_lines["bid_line"])
                d.line([top, bottom], fill=line_color, width=10)
            else:
                line_color = (255, 255, 0)
                top = (200, start_lines["ask_line"])
                bottom = (200, end_lines["bid_line"])
                d.line([top, bottom], fill=line_color, width=10)

            self.draw_profit_bar(img2, d, profit)


        elif position in ["buy", "sell"] and self.prev_trade[3] == 0:
            # print(f"Draw buy bar \n\nElse{position}\n\n:")
            #draw main vertical bar
            self.draw_current_trade(img2, d, "close")

            line_color = (255,0,255)
            top = (200, start_lines["ask_line"])
            bottom = (200, end_lines["bid_line"])
            d.line([top, bottom], fill=line_color, width=15)
            #draw end bar
            line_color = (255,0,255)
            top = (200, end_lines["ask_line"])
            bottom = (200, end_lines["bid_line"])
            d.line([top, bottom], fill=line_color, width=30) 
            #Draw start marker and line
            line_color = (255,0,255)
            top = (200, start_lines["ask_line"])
            bottom = (200, start_lines["bid_line"])
            d.line([top, bottom], fill=line_color, width=30)

            line_color = (2, 8, 135)
            top = (50, start_lines["ask_line"])
            bottom = (50, start_lines["bid_line"])
            d.line([top, bottom], fill=line_color, width=250)

            # self.draw_profit_bar(img2, d, profit)

            if self.prev_trade[2] == 2 and profit > 0:   
                #print right bar for the trade that has closed
                line_color = (0, 255, 0)
                top = (213, start_lines["ask_line"])
                bottom = (213, end_lines["bid_line"])
                d.line([top, bottom], fill=line_color, width=10) 

                line_color = (241, 254, 198)
                top = (190, start_lines["ask_line"])
                bottom = (190, end_lines["bid_line"])
                d.line([top, bottom], fill=line_color, width=10)             

            elif self.prev_trade[2] == 2 and profit < 0:
                line_color = (0, 255, 0)
                top = (213, start_lines["ask_line"])
                bottom = (213, end_lines["bid_line"])
                d.line([top, bottom], fill=line_color, width=10) 

                line_color = (255,0,0)
                top = (190, start_lines["ask_line"])
                bottom = (190, end_lines["bid_line"])
                d.line([top, bottom], fill=line_color, width=10)    
            elif self.prev_trade[2] == 1 and profit > 0:
                # print(f"\n\nHey I was here\n\n{self.prev_trade}")
                
                line_color = (255, 255, 0)
                top = (213, start_lines["ask_line"])
                bottom = (213, end_lines["bid_line"])
                d.line([top, bottom], fill=line_color, width=10) 

                line_color = (241, 254, 198)
                top = (190, start_lines["ask_line"])
                bottom = (190, end_lines["bid_line"])
                d.line([top, bottom], fill=line_color, width=10)    

            elif self.prev_trade[2] == 1 and profit < 0:
                line_color = (255, 255, 0)
                top = (213, start_lines["ask_line"])
                bottom = (213, end_lines["bid_line"])
                d.line([top, bottom], fill=line_color, width=10) 

                line_color = (255,0,0)
                top = (190, start_lines["ask_line"])
                bottom = (190, end_lines["bid_line"])
                d.line([top, bottom], fill=line_color, width=10) 
        else :#:position == "close" and self.prev_trade[2] == 0:
            self.draw_current_trade(img2, d, "close")
            # print("I run for position Close")
            line_color = (255,0,255)
            top = (200, end_lines["ask_line"])
            bottom = (200, end_lines["bid_line"])
            d.line([top, bottom], fill=line_color, width=30)  

        self.draw_profit_bar(img2, d, profit, account_balance, position)
        return img2


In [5]:
class ForexCustomEnv:
    def __init__(self, m1short, m5short, m1long, m5long, chart_decorator):
        # super(BaseEnv, self).__init__()
        self.m1short = m1short
        self.m5short = m5short
        self.m1long = m1long
        self.m5long = m5long
        self.trade_queue_m5_short = deque(maxlen=4)
        self.image_queue_m5_short = deque(maxlen=4)
        self.image_queue_m5_long = deque(maxlen=4)
        self.trade_queue_m5_long = deque(maxlen=4)
        self.dataset_directory = "M1M5Charts/"#"colabM1M5/episode2/"
        self.account_balance = 1000
        self.env_draw = False
        self.chart_decorator = chart_decorator
        
        self.prev_trade = deque(maxlen=4)
        self.prev_trade.append(0)
        self.prev_trade.append(0)
        self.prev_trade.append(0)
        self.prev_trade.append(0)
        
        self.current_step = 4
        self.action = 0
        self.position = "close"
        self.reward_ma = deque(maxlen=3)
        self.reward_Q = deque(maxlen=2)
        self.observation_space = (1,1,224,224,3)
        self.space = (1,1,224,224,3)
        self.action_space = 3
        
        # self.scaler = MinMaxScaler(feature_range=(0, 1)).fit(np.array([0,0,20]).reshape(-1,1))
    #base env compatibility
    def try_reset(self):
        return self.reset()
    def get_ask_bid_lines(self, image_path):
        ask_bid_lines = {"ask_line":None, "bid_line":None, "key": 0}
        img = Image.open(f"{self.dataset_directory}{image_path}")
        img_data = np.array(img)
        last_part = Image.fromarray(img_data[:,200:201,:])
        img1_pixels = last_part.load()
        ask_color = (255,0,0)
        bid_color = (119, 136, 153)
        for y in range(224):
            if img1_pixels[0,y] == ask_color:
                ask_bid_lines["ask_line"] = y
            if img1_pixels[0,y] == bid_color:
                ask_bid_lines["bid_line"] = y
        if ask_bid_lines["ask_line"] == None:
            # print("Gotcha")
            ask_bid_lines["ask_line"] = ask_bid_lines["bid_line"] + 3
            ask_bid_lines["bid_line"] = ask_bid_lines["bid_line"] + 6
            
        return ask_bid_lines , img        
    
    def reset(self, current_step = 4, step = 50):
        self.current_step = current_step
        self.step_size = step
        # print(f"DBG reset {self.m1short.iloc[self.current_step - 4]['image_path']}")

        linesq_m5_short, imageq_m5_short = self.get_ask_bid_lines(f"{self.m5short.iloc[self.current_step - 4]['image_path']}")
        linesq_m5_short["key"] = current_step - 4
        linest_m5_short, imaget_m5_short = self.get_ask_bid_lines(f"{self.m5short.iloc[self.current_step - 3]['image_path']}")
        linest_m5_short["key"] = current_step - 3
        liness_m5_short, images_m5_short = self.get_ask_bid_lines(f"{self.m5short.iloc[self.current_step - 2]['image_path']}")
        liness_m5_short["key"] = current_step - 2
        linesc_m5_short, imagec_m5_short = self.get_ask_bid_lines(f"{self.m5short.iloc[self.current_step - 1]['image_path']}")
        linesc_m5_short["key"] = current_step - 1
        
        linesq_m5_long, imageq_m5_long = self.get_ask_bid_lines(f"{self.m5long.iloc[self.current_step - 4]['image_path']}")
        linesq_m5_long["key"] = current_step - 4
        linest_m5_long, imaget_m5_long = self.get_ask_bid_lines(f"{self.m5long.iloc[self.current_step - 3]['image_path']}")
        linest_m5_long["key"] = current_step - 3
        liness_m5_long, images_m5_long = self.get_ask_bid_lines(f"{self.m5long.iloc[self.current_step - 2]['image_path']}")
        liness_m5_long["key"] = current_step - 2
        linesc_m5_long, imagec_m5_long = self.get_ask_bid_lines(f"{self.m5long.iloc[self.current_step - 1]['image_path']}")
        linesc_m5_long["key"] = current_step - 1

        self.trade_queue_m5_short.append(linesq_m5_short)
        self.trade_queue_m5_short.append(linest_m5_short)
        self.trade_queue_m5_short.append(liness_m5_short)
        self.trade_queue_m5_short.append(linesc_m5_short)
        self.image_queue_m5_short.append(imageq_m5_short)
        self.image_queue_m5_short.append(imaget_m5_short)
        self.image_queue_m5_short.append(images_m5_short)
        self.image_queue_m5_short.append(imagec_m5_short)         
        
        self.trade_queue_m5_long.append(linesq_m5_long)
        self.trade_queue_m5_long.append(linest_m5_long)
        self.trade_queue_m5_long.append(liness_m5_long)
        self.trade_queue_m5_long.append(linesc_m5_long)
        self.image_queue_m5_long.append(imageq_m5_long)
        self.image_queue_m5_long.append(imaget_m5_long)
        self.image_queue_m5_long.append(images_m5_long)
        self.image_queue_m5_long.append(imagec_m5_long)        
        
        self.done = 0
        self.reward = 0
        self.position = "close"
        self.reset_current_trade()
        
        next_state = self.get_obs()
        self.episode_profit = 0
        self.prev_trade = deque(maxlen=4)
        self.prev_trade.append(0)
        self.prev_trade.append(0)
        self.prev_trade.append(0)
        self.prev_trade.append(0)
        self.action = 0
        self.reward_ma = deque(maxlen=3)
        self.reward_Q = deque(maxlen=3)

        #reset standard limit in chart decorator
        self.chart_decorator.standard_limit = self.account_balance
        
        return next_state, self.reward, self.done, self.current_trade 
        
    def reset_current_trade(self ):
        self.current_trade = {}
        self.current_trade["current_step"] = self.current_step
        self.current_trade["ask_bid_lines_m5_short"] = None
        self.current_trade["ask_bid_lines_m5_long"] = None
        self.current_trade["entry_price"] = None
        self.current_trade["current_price"] = None
        self.current_trade["timesteps"] = 0
        self.current_trade["position"] = "close"
        self.current_trade["profit"] = 0
        self.current_trade["parent_ask_bid_lines_m5_short"] = None
        self.current_trade["parent_ask_bid_lines_m5_long"] = None
        self.current_trade["reward"] = 0
        self.current_trade["balance"] = self.account_balance
        self.current_trade["episode_profit"] = 0
        #this code is to add dimensions to 8 of them 4 for m1short 4 for m5short remaining to draw on m5 charts
        self.current_trade["ask_bid_lines_m5_short"] = None
        
        self.current_trade["highest"] = 0
        self.current_trade["price_ma"] = deque(maxlen=3)
        self.current_trade["reward_ma"] = deque(maxlen=3)
        self.account_balance = 1000
        self.old_balance = 1000
        
        
    def display_stacked_horizontally(self,imageq, imaget,images,imagec):
        # # Calculate the required dimensions for the new image
        new_width = imageq.width * 4
        new_height = imageq.height
        # # Create a new image with the required dimensions
        new_image = Image.new("RGB", (new_width, new_height))
        # # Paste the individual images side by side
        new_image.paste(imageq, (0, 0))
        new_image.paste(imaget, (imageq.width, 0))
        new_image.paste(images, (imageq.width + imaget.width, 0))
        new_image.paste(imagec, (imageq.width + imaget.width + images.width, 0))
        

    
    def reset_collection(self, t_q, i_q):
        _queue = deque(maxlen=4)
        for i in range(0, 4):
            _lines_c, _image_c = self.get_ask_bid_lines(f"{self.dataset_directory}{self.m1short.iloc[t_q[i]['key']]['image_path']}")
            _image_c = self.draw_buy_bar(_image_c, _lines_c, _lines_c, position="close", profit = 0)
            _queue.append(_image_c)
        return t_q, _queue
    
    def draw_collection(self,q, q2):
        # # Calculate the required dimensions for the new image
        new_width = q[0].width * 4
        new_height =q[0].height
        # # Create a new image with the required dimensions
        new_image = Image.new("RGB", (new_width, new_height))
        # # Paste the individual images side by side
        # new_image.paste(q[0], (0, 0))
        # new_image.paste(q2[1], (0, 0))
        # new_image.paste(q2[2], (q[0].width, 0))
        new_image.paste(q2[3], (q[0].width + q[0].width, 0))
        # new_image.paste(q[3], (q[0].width + q[0].width + q[0].width, 0))
        display(q2[3])
        
    def draw_buy_bar(self, current_image, start_lines, end_lines, position = None, profit = 0, prev_trade=None):
        # self.position = position
        # self.profit = profit
        return self.chart_decorator.draw_buy_bar(current_image, start_lines, end_lines, self.position, profit, prev_trade, self.account_balance)

    
    '''
    @description: Enters a trade in the forex environment
                  Enters the trade in the current time step, sets the position of the trade. sets the ask bid lines for the current frame
                  sets the entry price for the trade, and profit = 0. Then draws on the image the entry icon on the chart
                  and saves the updated current_image with modifications to the image queue with its corresponding ask and bid lines. 
                  then initializes the timesteps of the current trade to 0. sets the reward to 0.2
    @params
                  @position = None the position of the current_trade
                  @ask_bid_lines of the current_image
                  @current_image the current frame of the environment
                  
    '''
    


    # def softplus_reward(self, profit, timestep, highest=0,position = "close", threshold=0.8):

    #     loss = min(profit,0)
    #     if profit == 0: 
    #         profit = 1
    #     if highest == 0:
    #         highest = 1
    #     if profit < 0:
    #         return -np.log(abs(profit)) #,-np.log(abs(profit))/10+np.exp(-timestep/10)
    #     return np.log(profit) #+ np.log( profit )-np.log(highest)

    # def get_reward(self, c_trade):

    #     return self.softplus_reward(c_trade["profit"], c_trade["timesteps"],  c_trade["highest"], c_trade["position"])

    
    
    def enter_trade(self, ask_bid_lines_m1_short=None, ask_bid_lines_m5_short = None, ask_bid_lines_m1_long = None, ask_bid_lines_m5_long = None, current_image_m1_short=None, current_image_m5_short=None, current_image_m1_long=None, current_image_m5_long = None):
        self.current_trade["position"] = self.position
        
        self.current_trade["ask_bid_lines_m5_short"] = ask_bid_lines_m5_short
        self.current_trade["ask_bid_lines_m5_long"] = ask_bid_lines_m5_long
        self.current_trade["entry_price"] = float(f"{self.m5short.iloc[self.current_step]['Ask']}") if self.position == "buy" else float(f"{self.m5short.iloc[self.current_step]['Bid']}")
        self.current_trade["profit"] = 0
        self.current_trade["current_price"] = float(f"{self.m5short.iloc[self.current_step]['Ask']}")
        self.current_trade["timesteps"] = 1
        
        self.current_trade["parent_ask_bid_lines_m5_short"] = ask_bid_lines_m5_short
        self.current_trade["parent_ask_bid_lines_m5_long"] = ask_bid_lines_m5_long
        self.current_trade["balance"] = self.account_balance
        self.current_trade["highest"] = 0
        self.current_trade["price_ma"] = deque(maxlen=2)
        self.current_trade["reward_ma"] = deque(maxlen=2)
        #self.current_trade["highest"] += 5
        
        self.current_trade["price_ma"].append(0)
        self.current_trade["price_ma"].append(0)
        self.current_trade["price_ma"].append(0)

        
        self.current_trade["reward_ma"].append(0.10)
        self.current_trade["reward_ma"].append(0.10)
        self.current_trade["reward_ma"].append(0.10)

        
        current_image_m5_short = self.draw_buy_bar(current_image_m5_short, ask_bid_lines_m5_short, ask_bid_lines_m5_short, self.position , self.current_trade["profit"], self.prev_trade )
        current_image_m5_long = self.draw_buy_bar(current_image_m5_long, ask_bid_lines_m5_long, ask_bid_lines_m5_long, self.position, self.current_trade["profit"], self.prev_trade)
        self.trade_queue_m5_short.append(ask_bid_lines_m5_short)
        self.trade_queue_m5_long.append(ask_bid_lines_m5_long)
        self.image_queue_m5_short.append(current_image_m5_short)  
        self.image_queue_m5_long.append(current_image_m5_long)
        
        if self.env_draw:
            
            self.draw_collection(self.image_queue_m5_short,self.image_queue_m5_long) 
            # self.draw_collection(self.image_queue_m5_short,self.image_queue_m5_long) 
        self.reward = 0# self.get_reward(self.current_trade) #self.account_balance/self.account_balance
        self.reward_Q.append(self.reward)
        self.reward_Q.append(self.reward)
        self.reward_Q.append(self.reward)
        
        self.current_trade["reward"]  = self.reward
        # print(f"DBG: enter trade_ current_trade {self.current_trade}")
        
    def enter_idle_mode(self, ask_bid_lines_m1_short=None, ask_bid_lines_m5_short = None, ask_bid_lines_m1_long = None, ask_bid_lines_m5_long = None, current_image_m1_short=None, current_image_m5_short=None, current_image_m1_long=None, current_image_m5_long = None):
        
        self.current_trade["ask_bid_lines_m5_short"] = ask_bid_lines_m5_short
        self.current_trade["ask_bid_lines_m5_long"] = ask_bid_lines_m5_long
        self.current_trade["entry_price"] = float(f"{self.m5short.iloc[self.current_step]['Ask']}")
        self.current_trade["current_price"] = float(f"{self.m5short.iloc[self.current_step]['Ask']}")
        self.current_trade["position"] = self.position
        self.current_trade["reward"] = 0
        self.current_trade["timesteps"] = 1
        self.current_trade["profit"] = -3
        self.current_trade["parent_ask_bid_lines_m5_short"] = None
        self.current_trade["parent_ask_bid_lines_m5_long"] = None
        self.current_trade["balance"] = self.account_balance
        
        self.current_trade["price_ma"].append(0.10)
        self.current_trade["price_ma"].append(0.10)
        self.current_trade["price_ma"].append(0.10)
        self.current_trade["price_ma"].append(0.10)
        
        self.current_trade["reward_ma"].append(0.00)
        self.current_trade["reward_ma"].append(0.00)
        self.current_trade["reward_ma"].append(0.00)
        self.current_trade["reward_ma"].append(0.00)
        
        current_image_m5_short = self.draw_buy_bar(current_image_m5_short, ask_bid_lines_m5_short, ask_bid_lines_m5_short, self.position, self.current_trade["profit"], self.prev_trade)
        current_image_m5_long = self.draw_buy_bar(current_image_m5_long, ask_bid_lines_m5_long, ask_bid_lines_m5_long, self.position, self.current_trade["profit"], self.prev_trade)
        self.trade_queue_m5_short.append(ask_bid_lines_m5_short)
        self.trade_queue_m5_long.append(ask_bid_lines_m5_long)
        self.image_queue_m5_short.append(current_image_m5_short)  
        self.image_queue_m5_long.append(current_image_m5_long)
        if self.env_draw:
            self.draw_collection(self.image_queue_m5_short,self.image_queue_m5_long) 
            # self.draw_collection(self.image_queue_m5_short,self.image_queue_m5_long) 
        self.reward = 0#self.get_reward(self.current_trade)
        self.current_trade["reward"] = self.reward
        # print(f"DBG: enter idle_ current_trade {self.current_trade}")
    def hold_position(self, ask_bid_lines_m1_short=None, ask_bid_lines_m5_short = None, ask_bid_lines_m1_long = None, ask_bid_lines_m5_long = None, current_image_m1_short=None, current_image_m5_short=None, current_image_m1_long=None, current_image_m5_long = None):
        
        self.current_trade["current_price"] = float(f"{self.m5short.iloc[self.current_step]['Bid']}") if self.position == "buy" else float(f"{self.m5short.iloc[self.current_step]['Ask']}")
        self.current_trade["profit"] = self.current_trade["current_price"] - self.current_trade["entry_price"] if self.position == "buy" else self.current_trade["entry_price"] - self.current_trade["current_price"]
        self.current_trade["timesteps"] += 1
        self.current_trade["balance"] = self.account_balance
        self.current_trade["highest"] = self.current_trade["profit"] if self.current_trade["profit"] > self.current_trade["highest"] else self.current_trade["highest"]
        #self.current_trade["highest"] -= 15

        current_image_m5_short = self.draw_buy_bar(current_image_m5_short, self.current_trade["parent_ask_bid_lines_m5_short"], ask_bid_lines_m5_short, self.position , self.current_trade["profit"], self.prev_trade)
        current_image_m5_long = self.draw_buy_bar(current_image_m5_long, self.current_trade["parent_ask_bid_lines_m5_long"], ask_bid_lines_m5_long, self.position, self.current_trade["profit"], self.prev_trade)
        self.trade_queue_m5_short.append(ask_bid_lines_m5_short)
        self.trade_queue_m5_long.append(ask_bid_lines_m5_long)
        self.image_queue_m5_short.append(current_image_m5_short)
        self.image_queue_m5_long.append(current_image_m5_long)
        #fix price variance with ma for each trade
        self.current_trade["price_ma"].append(self.current_trade["profit"])
        
        if len(self.current_trade["price_ma"]) < 2:
            if self.current_trade["profit"] < 0 and self.current_trade > -10:
                self.current_trade["price_ma"].append(1)
                self.current_trade["price_ma"].append(1)
                # self.current_trade["price_ma"].append(1)
                # self.current_trade["price_ma"].append(1)
                
            elif self.current_trade["price_ma"] > 0 and self.current_trade["profit"] > 0:
                self.current_trade["price_ma"].append(self.current_trade["profit"])
                self.current_trade["price_ma"].append(self.current_trade["profit"])
                # self.current_trade["price_ma"].append(self.current_trade["profit"])
                # self.current_trade["price_ma"].append(self.current_trade["profit"])
                # print(f"DBG: hold_position {self.current_trade['price_ma']}")
        
        if self.env_draw:
            self.draw_collection(self.image_queue_m5_short,self.image_queue_m5_long) 
            # self.draw_collection(self.image_queue_m5_short,self.image_queue_m5_long) 
        if self.current_trade["profit"] >= 0:
            self.reward = 0#self.get_reward(self.current_trade)#(self.account_balance + self.current_trade["profit"])/self.account_balance
            self.current_trade["reward"] = self.reward
        else:
            self.reward = 0#self.get_reward(self.current_trade)#-1 - (self.account_balance + self.current_trade["profit"])/self.account_balance
            self.current_trade["reward"] = self.reward
        # print(f"DBG::hold positio   n_ current_trade {self.current_trade}")
        self.current_trade["reward_ma"].append(self.reward)
        self.reward = 0#sum(self.current_trade["reward_ma"])/len(self.current_trade["reward_ma"])
        self.reward_Q.append(self.reward)
        self.reward = sum(self.reward_Q)/len(self.reward_Q)
        self.current_trade["reward"] = self.reward

        
    
    def close_position(self, ask_bid_lines_m1_short=None, ask_bid_lines_m5_short = None, ask_bid_lines_m1_long = None, ask_bid_lines_m5_long = None, current_image_m1_short=None, current_image_m5_short=None, current_image_m1_long=None, current_image_m5_long = None, prev_trade = None):
        # print(f"self.position {self.position}")
        self.current_trade["current_price"] = float(f"{self.m5short.iloc[self.current_step]['Bid']}") if self.position == "buy" else float(f"{self.m5short.iloc[self.current_step]['Ask']}")
        self.current_trade["profit"] = self.current_trade["current_price"] - self.current_trade["entry_price"] if self.position == "buy" else self.current_trade["entry_price"] - self.current_trade["current_price"]
        self.current_trade["highest"] = self.current_trade["profit"] if self.current_trade["profit"] > self.current_trade["highest"] else self.current_trade["highest"]
        #self.current_trade["highest"] += 5

        current_image_m5_short = self.draw_buy_bar(current_image_m5_short, self.parent_ask_bid_lines_m5_short, ask_bid_lines_m5_short, self.position, self.current_trade["profit"] , prev_trade)
        current_image_m5_long = self.draw_buy_bar(current_image_m5_long, self.parent_ask_bid_lines_m5_long, ask_bid_lines_m5_long, self.position, self.current_trade["profit"], prev_trade)
        self.trade_queue_m5_short.append(ask_bid_lines_m5_short)
        self.trade_queue_m5_long.append(ask_bid_lines_m5_long)
        self.image_queue_m5_short.append(current_image_m5_short)
        self.image_queue_m5_long.append(current_image_m5_long)
        self.account_balance += self.current_trade["profit"]
        #fix high variance for the price
        self.current_trade["price_ma"].append(self.current_trade["profit"])
        
        if self.env_draw:
            self.draw_collection(self.image_queue_m5_short,self.image_queue_m5_long) 
            # self.draw_collection(self.image_queue_m5_short,self.image_queue_m5_long) 
        if self.current_trade["profit"] >= 0:
            if self.current_trade["profit"] * 80 < self.current_trade["highest"]:
                self.reward = 1#np.log(self.account_balance) - np.log(self.current_trade["highest"]-self.current_trade["profit"])
            else:
                self.reward = 1#np.log(self.account_balance)#self.get_reward(self.current_trade)##(self.account_balance + self.current_trade["profit"])/self.account_balance
        else:
            self.reward =  -1#-np.log(2) - np.log(abs(self.current_trade["profit"] - 20))#self.get_reward(self.current_trade)#-1 - (self.account_balance + self.current_trade["profit"])/self.account_balance
        #self.trade_queue, self.image_queue = self.reset_collection(self.trade_queue,self.image_queue)
        # self.account_balance += self.current_trade["profit"]
        
        self.current_trade["reward_ma"].append(self.reward)
        # self.reward = sum(self.current_trade["reward_ma"])/len(self.current_trade["reward_ma"])
        self.current_trade["reward"] = self.reward        
        
        self.current_trade["current_timestep"] = self.current_step
        self.current_trade["timesteps"] += 1
        self.current_trade["position"] = "close"
        self.current_trade["balance"] = self.account_balance
        self.current_trade["episode_profit"] = self.episode_profit
        self.position = "close"
        self.parent_ask_bid = None
        
        # print(f"DBG: close trade_ current_trade {self.current_trade}")
    def step(self, action):
        self.reward = 0
        self.done = 0
        self.action = action
        
        ask_bid_lines_m5_short, current_image_m5_short = self.get_ask_bid_lines(f"{self.m5short.iloc[self.current_step]['image_path']}")
        ask_bid_lines_m5_long, current_image_m5_long = self.get_ask_bid_lines(f"{self.m5long.iloc[self.current_step]['image_path']}")
        ask_bid_lines_m5_short["key"] = self.current_step
        ask_bid_lines_m5_long["key"] = self.current_step
        
        
        
        if self.position == "close":
            if action == 1:
                self.position = "buy"
                self.prev_trade.append(1)
                self.parent_ask_bid_lines_m5_short = ask_bid_lines_m5_short
                self.parent_ask_bid_lines_m5_long = ask_bid_lines_m5_long
                
                self.enter_trade(ask_bid_lines_m5_short = ask_bid_lines_m5_short, ask_bid_lines_m5_long = ask_bid_lines_m5_long, current_image_m5_short = current_image_m5_short, current_image_m5_long = current_image_m5_long)
            elif action == 2:
                self.position = "sell"
                self.prev_trade.append(2)
                self.parent_ask_bid_lines_m5_short = ask_bid_lines_m5_short
                self.parent_ask_bid_lines_m5_long = ask_bid_lines_m5_long
                self.enter_trade(ask_bid_lines_m5_short = ask_bid_lines_m5_short, ask_bid_lines_m5_long = ask_bid_lines_m5_long, current_image_m5_short = current_image_m5_short, current_image_m5_long = current_image_m5_long)
            elif action == 0:
                self.position = "close"
                self.prev_trade.append(0)
                self.parent_ask_bid_lines_m5_short = ask_bid_lines_m5_short
                self.parent_ask_bid_lines_m5_long = ask_bid_lines_m5_long
                self.enter_idle_mode(ask_bid_lines_m5_short = ask_bid_lines_m5_short, ask_bid_lines_m5_long = ask_bid_lines_m5_long, current_image_m5_short = current_image_m5_short, current_image_m5_long = current_image_m5_long)
        elif self.position == "buy":
            if action == 1:
                self.prev_trade.append(1)
                self.hold_position(ask_bid_lines_m5_short = ask_bid_lines_m5_short, ask_bid_lines_m5_long = ask_bid_lines_m5_long, current_image_m5_short = current_image_m5_short, current_image_m5_long = current_image_m5_long)
            elif action in [2]:
                self.prev_trade.append(0)
                self.close_position(ask_bid_lines_m5_short = ask_bid_lines_m5_short, ask_bid_lines_m5_long = ask_bid_lines_m5_long, current_image_m5_short = current_image_m5_short, current_image_m5_long = current_image_m5_long, prev_trade = self.prev_trade)                
                #fix standard limit and standard balance on chart 
                if self.account_balance > int((2/3) * self.chart_decorator.balance_limit) + self.chart_decorator.standard_balance:
                    # print("\n\nReseting the account balance in closing buy trade using 2\n\n the if statement\n\n\n")
                    self.chart_decorator.standard_balance = self.account_balance
                # elif self.chart_decorator.standard_balance * (2/3) > self.account_balance :
                    # print("\n\nReseting the account balance in closing buy trade using 2\n\n the elif statement\n\n\n")
                    # self.chart_decorator.standard_balance -= self.chart_decorator.balance_limit  
                # self.reward = (self.account_balance + self.current_trade["profit"])/1000
                # self.current_trade["reward"] = (self.account_balance + self.current_trade["profit"])/1000
                    
            elif action in [0]:
                self.prev_trade.append(0)
                self.close_position(ask_bid_lines_m5_short = ask_bid_lines_m5_short, ask_bid_lines_m5_long = ask_bid_lines_m5_long, current_image_m5_short = current_image_m5_short, current_image_m5_long=current_image_m5_long, prev_trade = self.prev_trade)   
                
                #fix standard limit and standard balance on chart
                if self.account_balance > int((2/3) * self.chart_decorator.balance_limit) + self.chart_decorator.standard_balance:
                    # print("\n\nReseting the account balance in closing buy trade using 0\n\n the if statement\n\n\n")
                    self.chart_decorator.standard_balance = self.account_balance
                # elif self.chart_decorator.standard_balance * (2/3) > self.account_balance :
                    # print("\n\nReseting the account balance in closing buy trade using 0\n\n the elif statement\n\n\n")
                    # self.chart_decorator.standard_balance -= self.chart_decorator.balance_limit  
            
        elif self.position == "sell":
            if action == 2:
                self.prev_trade.append(2)
                self.hold_position(ask_bid_lines_m5_short = ask_bid_lines_m5_short, ask_bid_lines_m5_long = ask_bid_lines_m5_long, current_image_m5_short = current_image_m5_short, current_image_m5_long = current_image_m5_long)
            #this reward is to penalize it not to change the position without closing the position
            elif action in [1]:
                self.prev_trade.append(0)
                self.close_position(ask_bid_lines_m5_short = ask_bid_lines_m5_short, ask_bid_lines_m5_long = ask_bid_lines_m5_long, current_image_m5_short = current_image_m5_short, current_image_m5_long = current_image_m5_long, prev_trade = self.prev_trade)
                #fix standard limit and standard balance on chart
                if self.account_balance > int((2/3) * self.chart_decorator.balance_limit) + self.chart_decorator.standard_balance:
                    # print("\n\nReseting the account balance in closing sell trade using 1\n\n the if statement\n\n\n")
                    
                    self.chart_decorator.standard_balance = self.account_balance
                # elif self.chart_decorator.standard_balance * (2/3) > self.account_balance :
                    # print("\n\nReseting the account balance in closing sell trade using 1\n\n the elif statement\n\n\n")
                    # self.chart_decorator.standard_balance -= self.chart_decorator.balance_limit  
                # self.reward = (self.account_balance + self.current_trade["profit"])/1000
                # self.current_trade["reward"] = (self.account_balance + self.current_trade["profit"])/1000
            elif action == 0:
                self.prev_trade.append(0)
                self.close_position(ask_bid_lines_m5_short = ask_bid_lines_m5_short, ask_bid_lines_m5_long = ask_bid_lines_m5_long, current_image_m5_short = current_image_m5_short, current_image_m5_long = current_image_m5_long, prev_trade = self.prev_trade)
                # self.prev_trade.append(0)
                #fix standard limit and standard balance on chart
                if self.account_balance > int((2/3) * self.chart_decorator.balance_limit) + self.chart_decorator.standard_balance:
                    # print("\n\nReseting the account balance in closing sell trade using 0\n\n the if statement\n\n\n")
                    # print(f"Account Balance == {self.account_balance} chart decorator standard_balance {self.chart_decorator.standard_balance}")
                    self.chart_decorator.standard_balance = self.account_balance
                    # print(f"updated standard balance == {self.chart_decorator.standard_balance}")
                # elif self.chart_decorator.standard_balance * (2/3) > self.account_balance :
                    # print("\n\nReseting the account balance in closing sell trade using 0\n\n the elif statement\n\n\n")
                    # self.chart_decorator.standard_balance -= self.chart_decorator.balance_limit 
        self.current_step+= self.step_size
        print(f"self.current_step {self.current_step}")
        next_state = self.get_obs()
        # if  self.account_balance > self.old_balance + 100:
        #     self.old_balance = self.account_balance
        
        if self.account_balance < 950 or self.current_step > 9800:#:self.old_balance:
            self.done = 1
        # if self.current_step > 9800:
        #     self.done = 1
        # print(self.trade_queue)
        
        return next_state, self.reward, self.done, self.current_trade
    
    def get_obs(self):
        # stacked_obs =   np.expand_dims(np.stack([
        #                 #np.asarray(self.image_queue_m5_long[1])/255,
        #                 #np.asarray(self.image_queue_m5_long[2])/255,
        #                 np.asarray(self.image_queue_m5_long[3]),
        #                 # np.asarray(self.image_queue_m5_short[0])/255, 
        #                 # np.asarray(self.image_queue_m5_short[1])/255, 
        #                 # np.asarray(self.image_queue_m5_short[2])/255, 
        #                 # np.asarray(self.image_queue_m5_short[3])/255,
        #                 # np.asarray(self.image_queue[0])/255, 
        #                 # np.asarray(self.image_queue[1])/255, 
        #                 # np.asarray(self.image_queue[2])/255, 
        #                 # np.asarray(self.image_queue[3])/255
        #     ]), axis=0)
        # stacked_obs_m5 = np.expand_dims(np.stack([np.asarray(self.image_queue_m5[0])/255, np.asarray(self.image_queue_m5[1])/255, np.asarray(self.image_queue_m5[2])/255, np.asarray(self.image_queue_m5[3])/255]), axis=0)
        # stacked = np.concatenate([stacked_obs_m5, stacked_obs], axis=0)
        return np.asarray(self.image_queue_m5_long[3])
        # return np.expand_dims(np.asarray(self.image_queue_m5_long[3]), axis=0)


Model

In [84]:
import tensorflow as tf
import gym
import ale_py
from ray.rllib.env.wrappers.atari_wrappers import wrap_deepmind
from ncps.tf import CfC
import numpy as np
from ncps.datasets.tf import AtariCloningDatasetTF


# Not used in this example
class ConvBlock(tf.keras.models.Sequential):
    def __init__(self):
        super(ConvBlock, self).__init__(
            [
                tf.keras.Input((84, 84, 4)),
                tf.keras.layers.Lambda(
                    lambda x: tf.cast(x, tf.float32) / 255.0
                ),  # normalize input
                tf.keras.layers.Conv2D(
                    64, 5, padding="same", activation="relu", strides=2
                ),
                tf.keras.layers.Conv2D(
                    128, 5, padding="same", activation="relu", strides=2
                ),
                tf.keras.layers.Conv2D(
                    128, 5, padding="same", activation="relu", strides=2
                ),
                tf.keras.layers.Conv2D(
                    256, 5, padding="same", activation="relu", strides=2
                ),
                tf.keras.layers.GlobalAveragePooling2D(),
            ]
        )


class ImpalaConvLayer(tf.keras.models.Sequential):
    def __init__(self, filters, kernel_size, strides, padding="valid", use_bias=False):
        super(ImpalaConvLayer, self).__init__(
            [
                tf.keras.layers.Conv2D(
                    filters=filters,
                    kernel_size=kernel_size,
                    strides=strides,
                    padding=padding,
                    use_bias=use_bias,
                    kernel_initializer=tf.keras.initializers.VarianceScaling(
                        scale=2.0, mode="fan_out", distribution="truncated_normal"
                    ),
                ),
                tf.keras.layers.BatchNormalization(momentum=0.99, epsilon=0.001),
                tf.keras.layers.ReLU(),
            ]
        )


class ImpalaConvBlock(tf.keras.models.Sequential):
    def __init__(self):
        super(ImpalaConvBlock, self).__init__(
            [
                ImpalaConvLayer(filters=16, kernel_size=8, strides=4),
                ImpalaConvLayer(filters=32, kernel_size=4, strides=2),
                ImpalaConvLayer(filters=32, kernel_size=3, strides=1),
                tf.keras.layers.Flatten(),
                tf.keras.layers.Dense(units=256, activation="relu"),
            ]
        )


class ConvCfC(tf.keras.Model):
    def __init__(self, n_actions):
        super().__init__()
        self.conv_block = ImpalaConvBlock()
        self.td_conv = tf.keras.layers.TimeDistributed(self.conv_block)
        self.rnn = CfC(64, return_sequences=True, return_state=True)
        self.linear = tf.keras.layers.Dense(n_actions)

    def get_initial_states(self, batch_size=1):
        return self.rnn.cell.get_initial_state(batch_size=batch_size, dtype=tf.float32)

    def call(self, x, training=None, **kwargs):
        has_hx = isinstance(x, list) or isinstance(x, tuple)
        initial_state = None
        if has_hx:
            # additional inputs are passed as a tuple
            x, initial_state = x

        x = self.td_conv(x, training=training)
        x, next_state = self.rnn(x, initial_state=initial_state)
        x = self.linear(x)
        if has_hx:
            return (x, next_state)
        return x

import matplotlib.pyplot as plt

def run_closed_loop(model, env, num_episodes=None):
    obs = env.reset()
    hx = model.get_initial_states()
    returns = []
    total_reward = 0
    while True:
        # add batch and time dimension (with a single element in each)
        plt.imshow(obs)
        plt.axis('off')
        plt.show()
        obs = np.expand_dims(np.expand_dims(obs, 0), 0)
        pred, hx = model.predict((obs, hx), verbose=0)
        action = pred[0, 0].argmax()
        # remove time and batch dimension -> then argmax
        obs, r, done, _ = env.step(action)
        print("reward ", r)
        total_reward += r
        if done:
            returns.append(total_reward)
            total_reward = 0
            obs = env.reset()
            hx = model.get_initial_states()
            # Reset RNN hidden states when episode is over
            if num_episodes is not None:
                # Count down the number of episodes
                num_episodes = num_episodes - 1
                if num_episodes == 0:
                    return returns


class ClosedLoopCallback(tf.keras.callbacks.Callback):
    def __init__(self, model, env):
        super().__init__()
        self.model = model
        self.env = env

    def on_epoch_end(self, epoch, logs=None):
        r = run_closed_loop(self.model, self.env, num_episodes=10)
        print(f"\nEpoch {epoch} return: {np.mean(r):0.2f} +- {np.std(r):0.2f}")


# if __name__ == "__main__":
#     # env = gym.make("ALE/Breakout-v5")
#     # env = wrap_deepmind(env)

#     # data = AtariCloningDatasetTF("breakout")
#     # batch_size = 32
#     # trainloader = data.get_dataset(32, split="train")
#     # valloader = data.get_dataset(32, split="val")

#     model = ConvCfC(3)

#     model.compile(
#         loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#         optimizer=tf.keras.optimizers.Adam(0.0001),
#         metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
#     )
    
#     model.build((None, None, 224, 224, 3))
#     model.summary()
#     # model.load_weights("models/breakout/model_ALE_Breakout_Impala.h5")
#     model.fit(
#         (x,y),
#         epochs=10,
#         validation_data=(x,y),
#         callbacks=[ClosedLoopCallback(model, env)],
#     )

    # Visualize Atari game and play endlessly
    # env = gym.make("ALE/Breakout-v5", render_mode="human")
    # env = wrap_deepmind(env)
    # run_closed_loop(model, env)

In [8]:
# x = np.array(X)
# y = np.array(y)

# # Reshape x to have the shape (num_samples, width, height, channels)
# x = x.reshape(-1, 224, 224, 3)
# x = np.expand_dims(x,axis=0)
# x = np.expand_dims(x,axis=0)
# y = np.expand_dims(y,axis=0)
# y = np.expand_dims(y,axis=0)

# print(y.shape)
# x.shape
# # (1, 1, 32, 84, 84, 4)
# #(1, 1, 32)

In [ ]:
m5short_step_4 = pd.read_csv("final_complete_m5short_timestep_4.csv")
m1short_step_4 = pd.read_csv("final_complete_m1short_timestep_4.csv")
m1long_step_4 = pd.read_csv("final_complete_m1long_timestep_4.csv")
m5long_step_4 = pd.read_csv("final_complete_m5long_timestep_4.csv")

chart_decorator = ChartDecorator()
env = ForexCustomEnv(m1short_step_4 ,m5short_step_4, m1long_step_4, m5long_step_4,chart_decorator)
env.env_draw = 1
counter = 4
state, reward, done, _ =  env.reset(counter, step= 50)
a = 0
X = []
Y = []
while not done:
    if counter <= 28:
        a = 2
    if counter == 29:
        a = 0
    if counter >= 30:
        a = 1
    if counter == 71:
        a = 0
    if counter >= 72 and counter < 99:
        a = 2
    if counter == 96:
        a = 0
    if counter >= 97 and counter <= 109:
        a = 1
    if counter == 110:
        a = 0
    if counter >= 111 and counter <= 175:
        a = 2
    if counter >= 176:
        a = 0

    if counter >= 177 and counter <= 184:
        a = 1
    if counter == 185:
        a = 0
    if counter >= 186:
        a = 2
    if counter == 199:
        a = 0
    # if counter >= 114 or counter <= 151:
    #     a = 2
    # if counter >= 152:
    #     a = 0
    action, pred = a, []
    next_state, reward, done, current_trade = env.step(action)
    print(counter)#,action, reward, done)
    # print(state)
    # print(f"next state {next_state}")
    counter += 1
    # print(state.shape)
    X.append(state)
    Y.append(action)

    state = next_state
    # break
    if counter > 250:
        break
    


In [62]:

x = np.array(X)
y = np.array(Y)

# Reshape x to have the shape (num_samples, width, height, channels)
x = x.reshape(-1, 224, 224, 3)
x = np.expand_dims(x,axis=0)
x = np.expand_dims(x,axis=0)
y = np.expand_dims(y,axis=0)
y = np.expand_dims(y,axis=0)


# Reshape x to have the shape (num_samples, width, height, channels)
# x = x.reshape(-1, 224, 224, 3)


In [37]:
int(np.random.random(1)*10) >= 9

False

In [43]:
(int(np.random.random(1)*10) >= 9)

True

In [46]:
np.random.choice([True,False])

True

In [65]:
import tensorflow as tf

def create_dataset(x, y):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    return dataset


x = tf.convert_to_tensor(x)
y = tf.convert_to_tensor(y)
# # Determine the dataset size based on the first dimension of x or y
dataset_size = tf.shape(x)[0] if tf.shape(x)[0] > tf.shape(y)[0] else tf.shape(y)[0]

# Create a range of indices for batching
indices = tf.range(dataset_size)

# Create a dataset from the indices
dataset = tf.data.Dataset.from_tensor_slices(indices)

# Apply a map function to retrieve elements from x and y based on the indices
dataset = dataset.map(lambda i: (x[i], y[i]))

# Prefetch the dataset for optimization
prefetched_dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

element_spec = prefetched_dataset

element_spec


<PrefetchDataset element_spec=(TensorSpec(shape=(1, 196, 224, 224, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(1, 196), dtype=tf.int64, name=None))>

In [ ]:
model = ConvCfC(3)

model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.0001),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.build((None, None, 224, 224, 3))
model.summary()
# model.load_weights("models/breakout/model_ALE_Breakout_Impala.h5")
model.fit(
    element_spec,
    epochs=50,
    validation_data=element_spec,
    #callbacks=[ClosedLoopCallback(model, env)],
)

In [ ]:

total_profits = 0
def run_closed_loop(model, env, total_profits,num_episodes=None):
    counter = 4
    env.env_draw = 1
    obs,_,_,current_trade = env.reset(counter, step=50)
    hx = model.get_initial_states()
    returns = []
    total_reward = 0
    
    while True:
        # add batch and time dimension (with a single element in each)
        # plt.imshow(obs)
        # plt.axis('off')
        # plt.show()
        obs = np.expand_dims(obs, axis=0)
        obs = np.expand_dims(obs, axis=0)
        # print(obs.shape)
        obs = tf.convert_to_tensor(obs)
        pred, hx = model.predict((obs, hx), verbose=0)
        action = pred[0, 0].argmax()
        # remove time and batch dimension -> then argmax
        obs, r, done, current_trade = env.step(action)
        # print("reward ", r)
        total_reward += r
        counter+=1
        if current_trade["position"] == "close":
            print("close found")
            total_profits += current_trade["profit"]
        print(counter , action, total_profits, current_trade["current_step"]) 
        if done:
            returns.append(total_reward)
            total_reward = 0
            obs = env.reset(4, step = 50)
            hx = model.get_initial_states()
            # Reset RNN hidden states when episode is over
            if num_episodes is not None:
                # Count down the number of episodes
                num_episodes = num_episodes - 1
                if num_episodes == 0:
                    return returns
chart_decorator = ChartDecorator()
env = ForexCustomEnv(m1short_step_4 ,m5short_step_4, m1long_step_4, m5long_step_4,chart_decorator)
total_profits = run_closed_loop(model, env, total_profits, 1)

Reinforcement Learning


In [70]:
import gym
from ray.tune.registry import register_env
from ray.rllib.models import ModelCatalog
from ray.rllib.algorithms.ppo import PPO
import time
import ale_py
from ray.rllib.env.wrappers.atari_wrappers import wrap_deepmind
import numpy as np

from ray.rllib.models.modelv2 import ModelV2
from ray.rllib.models.tf.recurrent_net import RecurrentNetwork
from ray.rllib.utils.annotations import override

import tensorflow as tf
from ncps.tf import CfC

In [98]:
class ConvCfCModel(RecurrentNetwork):
    """Example of using the Keras functional API to define a RNN model."""

    def __init__(
        self,
        obs_space,
        action_space,
        num_outputs,
        model_config,
        name,
        cell_size=64,
    ):
        super(ConvCfCModel, self).__init__(
            obs_space, action_space, num_outputs, model_config, name
        )
        self.cell_size = cell_size

        # Define input layers
        input_layer = tf.keras.layers.Input(
            shape=(None, obs_space.shape[0] * obs_space.shape[1] * obs_space.shape[2]),
            name="inputs",
        )
        state_in_h = tf.keras.layers.Input(shape=(cell_size,), name="h")
        seq_in = tf.keras.layers.Input(shape=(), name="seq_in", dtype=tf.int32)

        # Preprocess observation with a hidden layer and send to CfC
        self.conv_block = tf.keras.models.Sequential(
            [
                tf.keras.Input(
                    (obs_space.shape[0] * obs_space.shape[1] * obs_space.shape[2])
                ),  # batch dimension is implicit
                tf.keras.layers.Lambda(
                    lambda x: tf.cast(x, tf.float32) / 255.0
                ),  # normalize input
                tf.keras.layers.Reshape(
                    (obs_space.shape[0], obs_space.shape[1], obs_space.shape[2])
                ),
                tf.keras.layers.Conv2D(
                    64, 5, padding="same", activation="relu", strides=2
                ),
                tf.keras.layers.Conv2D(
                    128, 5, padding="same", activation="relu", strides=2
                ),
                tf.keras.layers.Conv2D(
                    128, 5, padding="same", activation="relu", strides=2
                ),
                tf.keras.layers.Conv2D(
                    256, 5, padding="same", activation="relu", strides=2
                ),
                tf.keras.layers.GlobalAveragePooling2D(),
            ]
        )
        self.td_conv = tf.keras.layers.TimeDistributed(self.conv_block)

        dense1 = self.td_conv(input_layer)
        cfc_out, state_h = CfC(
            cell_size, return_sequences=True, return_state=True, name="cfc"
        )(
            inputs=dense1,
            mask=tf.sequence_mask(seq_in),
            initial_state=[state_in_h],
        )

        # Postprocess CfC output with another hidden layer and compute values
        logits = tf.keras.layers.Dense(
            self.num_outputs, activation=tf.keras.activations.linear, name="logits"
        )(cfc_out)
        values = tf.keras.layers.Dense(1, activation=None, name="values")(cfc_out)

        # Create the RNN model
        self.rnn_model = tf.keras.Model(
            inputs=[input_layer, seq_in, state_in_h],
            outputs=[logits, values, state_h],
        )
        self.rnn_model.summary()

    @override(RecurrentNetwork)
    def forward_rnn(self, inputs, state, seq_lens):
        model_out, self._value_out, h = self.rnn_model([inputs, seq_lens] + state)
        return model_out, [h]

    @override(ModelV2)
    def get_initial_state(self):
        return [
            np.zeros(self.cell_size, np.float32),
        ]

    @override(ModelV2)
    def value_function(self):
        return tf.reshape(self._value_out, [-1])



In [105]:
ModelCatalog.register_custom_model("cfc", ConvCfCModel)

def env_creator(x):
    m5short_step_4 = pd.read_csv("final_complete_m5short_timestep_4.csv")
    m1short_step_4 = pd.read_csv("final_complete_m1short_timestep_4.csv")
    m1long_step_4 = pd.read_csv("final_complete_m1long_timestep_4.csv")
    m5long_step_4 = pd.read_csv("final_complete_m5long_timestep_4.csv")
    chart_decorator = ChartDecorator()
    return ForexCustomEnv(m1short_step_4 ,m5short_step_4, m1long_step_4, m5long_step_4,chart_decorator)
register_env("forex_env", env_creator)

In [106]:

if __name__ == "__main__":
    print("Running")
    # parser = argparse.ArgumentParser()
    # parser.add_argument("--env", type=str, default="ForexCustom")
    # parser.add_argument("--cont", default="")
    # parser.add_argument("--render", action="store_true")
    # parser.add_argument("--hours", default=2, type=int)
    # args = parser.parse_args()
    class Args:
        pass
    args = Args()
    args.env = "forex_env"
    args.cont = ""
    args.render = ""
    args.hours = 2
    # register_env("atari_env", lambda env_config: wrap_deepmind(gym.make(args.env)))
    config = {
        "log_level":"INFO",
        "monitor":True,
        "env": "forex_env",
        "preprocessor_pref": None,
        "gamma": 0.99,
        "num_gpus": 0,
        "num_workers": 4,
        "num_envs_per_worker": 1,
        "create_env_on_driver": True,
        "lambda": 0.95,
        "kl_coeff": 0.5,
        "clip_rewards": True,
        "clip_param": 0.1,
        "vf_clip_param": 10.0,
        "entropy_coeff": 0.01,
        "rollout_fragment_length": 100,
        "sgd_minibatch_size": 500,
        "train_batch_size": 2000,
        "num_sgd_iter": 10,
        "recreate_failed_workers":True,
        "ignore_worker_failures":True,        
        "batch_mode": "truncate_episodes",
        "observation_filter": "NoFilter",
        "model": {
            "vf_share_layers": True,
            "custom_model": "cfc",
            "max_seq_len": 20,
            "custom_model_config": {
                "cell_size": 64,
            },
        },
        "framework": "tf2",
    }

    algo = PPO(config=config)

    os.makedirs(f"rl_ckpt/{args.env}", exist_ok=True)
    if args.cont != "":
        algo.load_checkpoint(f"rl_ckpt/{args.env}/{args.cont}")

    if args.render == 2222:
        run_closed_loop(
            algo,
            config,
        )
    else:
        print("Training STarted")
        start_time = time.time()
        last_eval = 0
        while True:
            info = algo.train()
            if time.time() - last_eval > 60 * 2:  # every 5 minutes print some stats
                print(f"Ran {(time.time()-start_time)/60/60:0.1f} hours")
                print(
                    f"    sampled {info['info']['num_env_steps_sampled']/1000:0.0f}k steps"
                )
                print(f"    policy reward: {info['episode_reward_mean']:0.1f}")
                last_eval = time.time()
                ckpt = algo.save_checkpoint(f"rl_ckpt/{args.env}")
                print(f"    saved checkpoint '{ckpt}'")

            elapsed = (time.time() - start_time) / 60  # in minutes
            if elapsed > args.hours * 60:
                break

Running


2023-05-21 23:21:17,367	INFO algorithm.py:2292 -- Executing eagerly (framework='tf2'), with eager_tracing=False. For production workloads, make sure to set eager_tracing=True  in order to match the speed of tf-static-graph (framework='tf'). For debugging purposes, `eager_tracing=False` is the best choice.
(RolloutWorker pid=42370) 2023-05-21 23:21:26,511	ERROR worker.py:763 -- Exception raised in creation task: The actor died because of an error raised in its creation task, ray::RolloutWorker.__init__() (pid=42370, ip=127.0.1.1, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x7f57d6dace50>)
(RolloutWorker pid=42370)   File "/home/silence/Projects/ForexLab/dataset/episode2/env10/lib/python3.10/site-packages/ray/rllib/utils/pre_checks/env.py", line 140, in check_gym_environments
(RolloutWorker pid=42370)     raise ValueError("Observation space must be a gym.space")
(RolloutWorker pid=42370) ValueError: Observation space must be a gym.space
(RolloutWorker pid=42370) 
(

ValueError: Traceback (most recent call last):
  File "/home/silence/Projects/ForexLab/dataset/episode2/env10/lib/python3.10/site-packages/ray/rllib/utils/pre_checks/env.py", line 77, in check_env
    check_gym_environments(env)
  File "/home/silence/Projects/ForexLab/dataset/episode2/env10/lib/python3.10/site-packages/ray/rllib/utils/pre_checks/env.py", line 140, in check_gym_environments
    raise ValueError("Observation space must be a gym.space")
ValueError: Observation space must be a gym.space

The above error has been found in your environment! We've added a module for checking your custom environments. It may cause your experiment to fail if your environment is not set up correctly. You can disable this behavior by setting `disable_env_checking=True` in your environment config dictionary. You can run the environment checking module standalone by calling ray.rllib.utils.check_env([env]).

In [107]:
!cat /home/silence/Projects/ForexLab/dataset/episode2/env10/lib/python3.10/site-packages/ray/rllib/utils/pre_checks/env.py

"""Common pre-checks for all RLlib experiments."""
from copy import copy
import inspect
import logging
import gym
import numpy as np
import traceback
import tree  # pip install dm_tree
from typing import TYPE_CHECKING, Set

from ray.actor import ActorHandle
from ray.rllib.utils.annotations import DeveloperAPI
from ray.rllib.utils.error import UnsupportedSpaceException
from ray.rllib.utils.spaces.space_utils import (
    convert_element_to_space_type,
    get_base_struct_from_space,
)
from ray.rllib.utils.typing import EnvType
from ray.util import log_once

if TYPE_CHECKING:
    from ray.rllib.env import BaseEnv, MultiAgentEnv

logger = logging.getLogger(__name__)


@DeveloperAPI
def check_env(env: EnvType) -> None:
    """Run pre-checks on env that uncover common errors in environments.

    Args:
        env: Environment to be checked.

    Raises:
        ValueError: If env is not an instance of SUPPORTED_ENVIRONMENT_TYPES.
        ValueError: See check_gym_env docstring for details.